# 01 Pytorch tensor
#### ＝＝＝ 目次 ＝＝＝
0. Pytorchの呼び出し
1. Tensorの生成
2. 基本演算
3. 誤差逆伝播&勾配降下法
4. GPUの使用

## Pytorchの特徴
- Numpyに代わってGPU上で動くパッケージ
- 柔軟性があり高速な深層学習のプラットフォーム
- define by run

---
## 0. Pytorchの呼び出し
`torch`という名前のモジュールをインポートする

In [1]:
import torch

---
## 1. Tensor(テンソル)の生成
tensor：多次元配列のようなもの(ex. ベクトル，行列)

Numpyのndarrayのようなもの(ndarrayと比べて，GPUを使うことで計算を高速化できる)

|<div align='center'>関数</div>|<div align='center'>意味</div>|<div align='center'>例</div>|
|---|---|---|
|<div align='left'>torch.tensor(array)</div>|<div align='left'>配列をtensorに変換</div>|<div align='left'>torch.tensor([2.5, 5.0, 3.6])</div>|
|<div align='left'>torch.empty(shape)</div>|<div align='left'>空のテンソルを作成 (何かしらの値が入っている)</div>|<div align='left'>torch.empty(2, 5)</div>|
|<div align='left'>torch.zeros(shape)</div>|<div align='left'>0のテンソルを作成</div>|<div align='left'>torch.zeros(2, 5)</div>|
|<div align='left'>torch.ones(shape)</div>|<div align='left'>1のテンソルを作成</div>|<div align='left'>torch.ones(2, 5)</div>|
|<div align='left'>torch.full(shape,fill_value)</div>|<div align='left'>任意の値のテンソルを作成</div>|<div align='left'>torch.full((2, 5),fill_value=4)</div>|
|<div align='left'>torch.zeros_like(tensor)</div>|<div align='left'>引数のテンソルと同じサイズの0のテンソルを作成</div>|<div align='left'>torch.zeros_like(a)</div>|
|<div align='left'>torch.eye(shape)</div>|<div align='left'>単位行列を作成</div>|<div align='left'>torch.eye(3, 3)</div>|
|<div align='left'>torch.rand(shape)</div>|<div align='left'>[0, 1]の一様分布による乱数</div>|<div align='left'>torch.rand(2, 5)</div>|
|<div align='left'>torch.randn(shape)</div>|<div align='left'>標準正規分布による乱数</div>|<div align='left'>torch.randn(2, 5)</div>|

`torch.tensor()`：配列をtensorに変換

`dtype`で値のデータ型を指定

In [2]:
# 一次元配列
x = torch.tensor([5.5, 3, 2.4])
print(x)

tensor([5.5000, 3.0000, 2.4000])


In [3]:
# 二次元配列
x = torch.tensor([[3, 5, 2],
                  [8, 8, 1],
                  [4, 1, 5],
                  [5, 8, 8],
                  [2, 5, 2]], dtype=torch.float)
print(x)

tensor([[3., 5., 2.],
        [8., 8., 1.],
        [4., 1., 5.],
        [5., 8., 8.],
        [2., 5., 2.]])


`torch.rand()`：[0, 1]の一様分布による乱数

In [4]:
x = torch.rand(2, 5)
print(x)

tensor([[0.6855, 0.6594, 0.2710, 0.3844, 0.8784],
        [0.2721, 0.9230, 0.8516, 0.2811, 0.5556]])


`torch.randn()`：標準正規分布による乱数 (平均0, 分散1)

In [5]:
x = torch.randn(2, 5)
print(x)

tensor([[-1.6088, -0.1886,  2.0373,  1.3687,  0.3790],
        [ 0.0522, -1.3260, -0.5854, -1.3858, -1.2349]])


---
## tensorのshape
`変数.shape` or `変数.size()`：tensorのshapeを返す

In [6]:
x = torch.randn(3, 6)
print(x.shape)
print(x.size())

torch.Size([3, 6])
torch.Size([3, 6])


`変数.view()`：tensorのshapeを変更したものを返す

In [7]:
x = torch.randn(4, 4)
y = x.view(16)
z = x.view(-1, 8) # -1を使うと自動で調整してくれる
print("x :", x.shape)
print("y :", y.shape)
print("z :", z.shape)

x : torch.Size([4, 4])
y : torch.Size([16])
z : torch.Size([2, 8])


---
## スライス
リストやndarrayのように，スライスを用いることで一部を抽出できる

In [8]:
x = torch.randn(4, 6)
print(x)
print(x[1:3, :])

tensor([[ 1.6004, -1.1912,  1.4197,  0.4917, -0.4241,  0.0186],
        [-0.9033,  0.6858,  0.7149, -1.6444, -0.2864,  0.7098],
        [ 0.8180, -1.1578, -1.0977,  1.1387, -0.4770, -0.9827],
        [-0.3123,  0.1478,  0.8375, -0.2153, -0.4981, -1.0739]])
tensor([[-0.9033,  0.6858,  0.7149, -1.6444, -0.2864,  0.7098],
        [ 0.8180, -1.1578, -1.0977,  1.1387, -0.4770, -0.9827]])


---
## Numpyとの変換
`変数.numpy()`：tensor → ndarray

In [9]:
a = torch.randn(2, 3)
print(type(a))
print(a)

b = a.numpy()
print(type(b))
print(b)

<class 'torch.Tensor'>
tensor([[-0.9772, -0.2097,  0.8352],
        [-0.3463, -1.0564,  0.2543]])
<class 'numpy.ndarray'>
[[-0.9771883  -0.20965306  0.8351917 ]
 [-0.34626698 -1.0563529   0.25432095]]


`torch.from_numpy(ndarray)`：ndarray → tensor

In [10]:
import numpy as np
a = np.ones(5)
print(type(a))
print(a)

b = torch.from_numpy(a)
print(type(b))
print(b)

<class 'numpy.ndarray'>
[1. 1. 1. 1. 1.]
<class 'torch.Tensor'>
tensor([1., 1., 1., 1., 1.], dtype=torch.float64)


---
## 2. 基本演算

|<div align='center'>演算</div>|<div align='center'>演算子</div>|
|---|---|
|<div align='center'>足し算</div>|<div align='center'>+</div>|
|<div align='center'>アダマール積</div>|<div align='center'>*</div>|
|<div align='center'>行列積</div>|<div align='center'>torch.mm()</div>|
|<div align='center'>要素の和</div>|<div align='center'>torch.sum()</div>|
|<div align='center'>要素の平均</div>|<div align='center'>torch.mean()</div>|
|<div align='center'>要素の標準偏差</div>|<div align='center'>torch.std()</div>|
|<div align='center'>要素の最大値</div>|<div align='center'>torch.max()</div>|

In [11]:
# テンソルの作成
x = torch.tensor([[4., 3.], 
                  [2., 1.]])
y = torch.tensor([[2., 2.], 
                  [1., 1.]])

In [12]:
# 足し算
x + y

tensor([[6., 5.],
        [3., 2.]])

In [13]:
# アダマール積
x * y

tensor([[8., 6.],
        [2., 1.]])

In [14]:
# 行列積
torch.mm(x, x)

tensor([[22., 15.],
        [10.,  7.]])

In [15]:
# 要素の和
torch.sum(x)

tensor(10.)

In [16]:
# 要素の最大値
torch.max(x)

tensor(4.)

In [17]:
torch.max(x, dim=1)

torch.return_types.max(
values=tensor([4., 2.]),
indices=tensor([0, 0]))

---
## 3. 誤差逆伝播&勾配降下法
`変数.backward()`：backpropagation(誤差逆伝播)による微分を行う．

`requires_grad=True`を指定することでtensorの勾配を保持する．

例.
$$y=w^2$$
$$\frac{dy}{dw} = 2w$$
$$\frac{dy}{dw}|_{w=1.0} = 2.0$$

In [18]:
w = torch.tensor(1.0, requires_grad=True)
y = w * w
print("w :", w)
print("y :", y)

w : tensor(1., requires_grad=True)
y : tensor(1., grad_fn=<MulBackward0>)


In [19]:
# backpropagation
y.backward()
print("wの勾配", w.grad)

wの勾配 tensor(2.)


## Optimizerによる勾配降下法
backpropagationで求めた勾配を用いて，optimizerによりtensorの値を更新する(学習)

例. $y=w^2$が最小となるときの$w$を勾配降下法で求める．(初期値 $w=1$)

In [20]:
# 初期値1のパラメータ
w = torch.tensor(1.0, requires_grad=True)
print(w)

tensor(1., requires_grad=True)


In [21]:
import torch.optim as optim

# optimizerを定義
# 更新するパラメータや学習率などを指定
optimizer = optim.SGD([w], lr=0.1)

SGDによる更新を1回行う

In [22]:
optimizer.zero_grad() # 勾配を初期化
y = w * w             # 順伝播
y.backward()          # backpropagation
optimizer.step()      # 勾配を元にパラメータを更新

print(w)

tensor(0.8000, requires_grad=True)


続けて更新を20回行う

In [23]:
# 20回パラメータを更新
for i in range(1, 21):
    optimizer.zero_grad() # 勾配を初期化
    y = w * w             # 順伝播
    y.backward()          # backpropagation
    optimizer.step()      # 勾配を元にパラメータを更新

    print(i, w)

1 tensor(0.6400, requires_grad=True)
2 tensor(0.5120, requires_grad=True)
3 tensor(0.4096, requires_grad=True)
4 tensor(0.3277, requires_grad=True)
5 tensor(0.2621, requires_grad=True)
6 tensor(0.2097, requires_grad=True)
7 tensor(0.1678, requires_grad=True)
8 tensor(0.1342, requires_grad=True)
9 tensor(0.1074, requires_grad=True)
10 tensor(0.0859, requires_grad=True)
11 tensor(0.0687, requires_grad=True)
12 tensor(0.0550, requires_grad=True)
13 tensor(0.0440, requires_grad=True)
14 tensor(0.0352, requires_grad=True)
15 tensor(0.0281, requires_grad=True)
16 tensor(0.0225, requires_grad=True)
17 tensor(0.0180, requires_grad=True)
18 tensor(0.0144, requires_grad=True)
19 tensor(0.0115, requires_grad=True)
20 tensor(0.0092, requires_grad=True)


$y=w^2$が最小となるときの$w$に近づいてることが分かる

---
## 4. GPUの使用
`torch.cuda.is_available()`：GPU(cuda)が使用できる場合`True`を，できない場合`False`を返す

In [24]:
torch.cuda.is_available()

False

使用しているデバイスをdeviceに代入する

cudaが使用可能であれば`"cuda:0"`，そうでなければ`"cpu"`を指定

In [25]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("使用デバイス：", device)

使用デバイス： cpu


`変数.to(device)`：変数をデバイス(cuda or cpu)に渡す

これによりデバイス上で計算が可能

In [26]:
x = torch.randn(4)
y = torch.randn(4)

# tensorをGPUへ
x = x.to(device)
y = y.to(device)

z = x + y # GPU上で計算が行われる
print(z)

tensor([-0.0784,  1.9486,  0.4547, -2.0565])


変数をCPUに渡す

In [27]:
z = z.to("cpu")
print(z)

tensor([-0.0784,  1.9486,  0.4547, -2.0565])


---
## マルチGPUの使用について
GPUが複数使用できる場合，`torch.nn.DataParallel`をモデルに適用することで並列計算が行える．

使用できるGPUの個数は`torch.cuda.device_count()`で確認できる．

In [28]:
# netを定義した後に記述
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs")
    net = nn.DataParallel(net)
    net.to(device)

ネットワークについては`02_pytorch_network.ipynb`で説明する．